In [1]:
import time
import os
import sys
import numpy as np
import ipympl
import matplotlib
#matplotlib.use('nbagg')
#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt

import astropy.units as u
from astropy import stats
from astropy.io import fits
from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT

In [2]:
%cd /Users/tim/MMT/wfsdat/20180201

/Users/tim/MMT/wfsdat/20180201


In [3]:
%ls *.fits.zernike

f9wfs_20180131-190219.fits.zernike  f9wfs_20180131-192541.fits.zernike
f9wfs_20180131-190336.fits.zernike  f9wfs_20180131-192731.fits.zernike
f9wfs_20180131-190443.fits.zernike  f9wfs_20180131-192927.fits.zernike
f9wfs_20180131-190620.fits.zernike  f9wfs_20180131-193109.fits.zernike
f9wfs_20180131-190736.fits.zernike  f9wfs_20180131-193325.fits.zernike
f9wfs_20180131-190910.fits.zernike  f9wfs_20180131-193509.fits.zernike
f9wfs_20180131-191110.fits.zernike  f9wfs_20180131-193702.fits.zernike
f9wfs_20180131-191321.fits.zernike  f9wfs_20180131-193901.fits.zernike
f9wfs_20180131-191546.fits.zernike  f9wfs_20180131-194155.fits.zernike
f9wfs_20180131-191734.fits.zernike  f9wfs_20180131-194349.fits.zernike
f9wfs_20180131-191925.fits.zernike  f9wfs_20180131-194553.fits.zernike
f9wfs_20180131-192131.fits.zernike  f9wfs_20180131-194806.fits.zernike
f9wfs_20180131-192350.fits.zernike


In [4]:
%more notes

f9wfs_20180131-191734.fits - 60 sec baseline after applying all M2 corrections
f9wfs_20180131-191925.fits - 60 sec baseline after applying all M2 corrections
f9wfs_20180131-192131.fits - 60 sec baseline after applying all M2 corrections
f9wfs_20180131-192350.fits - cleared forces from M1
f9wfs_20180131-192541.fits - "
f9wfs_20180131-192731.fits - "
f9wfs_20180131-192927.fits - apply null force set
f9wfs_20180131-193109.fits - "
f9wfs_20180131-193325.fits - "
f9wfs_20180131-193509.fits - "
f9wfs_20180131-193702.fits - clear forces
f9wfs_20180131-193901.fits - apply null forces


In [5]:
baseline1_files = ["f9wfs_20180131-191734.fits", "f9wfs_20180131-191925.fits", "f9wfs_20180131-192131.fits"]
baseline2_files = ["f9wfs_20180131-192350.fits", "f9wfs_20180131-192541.fits", "f9wfs_20180131-192731.fits"]
baseline3_files = [
    "f9wfs_20180131-192927.fits", "f9wfs_20180131-193109.fits", 
    "f9wfs_20180131-193325.fits", "f9wfs_20180131-193509.fits"
]

In [7]:
plt.close('all')
f9wfs = WFSFactory(wfs="newf9", nzern=37)

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


In [8]:
baseline1 = []
for f in baseline1_files:
    results = f9wfs.measure_slopes(f, 'blue', plot=True)
    zresults = f9wfs.fit_wavefront(results, plot=True)
    baseline1.append(zresults)
baseline2 = []
for f in baseline2_files:
    results = f9wfs.measure_slopes(f, 'blue', plot=True)
    zresults = f9wfs.fit_wavefront(results, plot=True)
    baseline2.append(zresults)
baseline3 = []
for f in baseline3_files:
    results = f9wfs.measure_slopes(f, 'blue', plot=True)
    zresults = f9wfs.fit_wavefront(results, plot=True)
    baseline3.append(zresults)

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecol

In [9]:
baseline3[0]['zernike'].fringe_bar_chart().show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [10]:
rms = []
mean = []
for baseline in [baseline1, baseline2, baseline3]:
    baseline_mean = ZernikeVector()
    for z in baseline:
        baseline_mean += z['zernike']
    baseline_mean /= len(baseline)
    baseline_rms = ZernikeVector()**2
    for z in baseline:
        baseline_rms += (z['zernike'] - baseline_mean)**2
    baseline_rms = baseline_rms**0.5
    rms.append(baseline_rms)
    mean.append(baseline_mean)

In [11]:
rms[2]

Fringe Coefficients
 Z02:       554 nm 	 X Tilt (1, 1)
 Z03:       494 nm 	 Y Tilt (1, -1)
 Z04:       944 nm 	 Defocus (2, 0)
 Z05:       623 nm 	 Primary Astig at 45° (2, -2)
 Z06:       290 nm 	 Primary Astig at 0° (2, 2)
 Z07:       347 nm 	 Primary Y Coma (3, -1)
 Z08:       370 nm 	 Primary X Coma (3, 1)
 Z09:       371 nm 	 Y Trefoil (3, -3)
 Z10:       164 nm 	 X Trefoil (3, 3)
 Z11:       287 nm 	 Primary Spherical (4, 0)
 Z12:       189 nm 	 Secondary Astigmatism at 0° (4, 2)
 Z13:       342 nm 	 Secondary Astigmatism at 45° (4, -2)
 Z14:       232 nm 	 X Tetrafoil (4, 4)
 Z15:       371 nm 	 Y Tetrafoil (4, -4)
 Z16:       664 nm 	 Secondary X Coma (5, 1)
 Z17:       367 nm 	 Secondary Y Coma (5, -1)
 Z18:       287 nm 	 Secondary X Trefoil (5, 3)
 Z19:       580 nm 	 Secondary Y Trefoil (5, -3)
 Z20:      91.6 nm 	 X Pentafoil (5, 5)
 Z21:       334 nm 	 Y Pentafoil (5, -5)
 Z22:       263 nm 	 Secondary Spherical (6, 0)
 Z23:       532 nm 	 Tertiary Astigmatism at 45° (6, 

In [8]:
plt.close('all')